In [1]:
!pip install streamlit transformers torch pandas matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [11]:
%%writefile app.py
import streamlit as st
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

# List of aspects
aspects = [
    "service", "staff", "crew", "attitude", "luggage", "baggage", "lost", "claim",
    "flight", "delay", "cancellation", "late", "boarding", "process", "gate", "check-in",
    "seat", "comfort", "legroom", "space", "food", "meal", "snack", "drink",
    "price", "cost", "value", "money", "cleanliness", "dirty", "hygiene",
    "entertainment", "wifi", "tv", "screen", "communication", "update", "notice",
    "ground service", "check in", "in-flight entertainment", "cabin crew", "flight path",
    "flight delay", "refund policy", "rebooking", "online check-in"
]

# Function to extract aspects
def extract_aspects(text, aspect_list):
    text = text.lower()
    matched = set()
    for aspect in aspect_list:
        pattern = r'\b' + re.escape(aspect.lower().replace('-', ' ')) + r's?\b'
        if re.search(pattern, text):
            matched.add(aspect)
    return list(matched)

# Load general sentiment model
@st.cache_resource
def load_general_model():
    return pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Load ABSA model & tokenizer
@st.cache_resource
def load_absa_model():
    tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
    model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
    return tokenizer, model

# Aspect-level sentiment
def analyze_aspect_sentiment(review, aspect, tokenizer, model):
    inputs = tokenizer(f"[CLS] {review} [SEP] {aspect} [SEP]", return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)[0].numpy()
    sentiment = ["negative", "neutral", "positive"][probs.argmax()]
    return sentiment, float(probs.max())

# Streamlit UI
st.title("🧠 Aspect-Based Sentiment Analyzer")

text = st.text_area("✏️ Enter a customer review:", height=200)

if st.button("🔍 Analyze"):
    if not text.strip():
        st.error("Please enter a review.")
    else:
        with st.spinner("Loading models and analyzing..."):
            general_model = load_general_model()
            tokenizer, absa_model = load_absa_model()

            # General Sentiment
            general_result = general_model(text)[0]
            st.subheader("Overall Sentiment")
            st.write(f"**Sentiment**: {general_result['label']} (Confidence: {general_result['score']:.4f})")

            # Aspect Detection
            aspects_found = extract_aspects(text, aspects)
            st.subheader("🔍 Detected Aspects")
            if aspects_found:
                for asp in aspects_found:
                    asp_sentiment, conf = analyze_aspect_sentiment(text, asp, tokenizer, absa_model)
                    st.write(f"**{asp}** → {asp_sentiment.capitalize()} (Confidence: {conf:.4f})")
            else:
                st.info("No known aspects found in the text.")

# Sidebar
st.sidebar.markdown("## ℹ️ How to use")
st.sidebar.write("1. Enter a review mentioning things like service, food, delay, etc.")
st.sidebar.write("2. The app detects aspects and returns specific sentiments.")


Overwriting app.py


In [3]:
!pip install streamlit transformers torch
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧
added 22 packages in 4s
⠧
⠧3 packages are looking for funding
⠧  run `npm fund` for details
⠧

In [4]:
# Your public ip is the password to the localtunnel
!curl ipv4.icanhazip.com

34.21.13.203


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.21.13.203:8501

⠴⠦⠧⠇⠏⠋your url is: https://mean-kids-win.loca.lt
2025-07-13 21:26:36.917216: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752441996.948376    3376 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752441996.957232    3376 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fall